# Exp M3: Fine λ Sweep - Critical Region (η=0.8)

## 目的
臨界点近傍（λ≈0.40-0.50）の詳細な構造を把握し、
variance peak（揺らぎの最大）を定量的に特定する。

## 実験設計
- **λ**: 0.38-0.50（0.02刻み、7点）
- **η（ノイズ率）**: 0.8のみ
- **Seeds**: 0-9（10個）← 揺らぎの統計を取るため
- **Total**: 7 × 10 = **70 runs**

## M1, M2, M3 の関係
- **M1**: 粗スイープで全体像を把握
- **M2**: 臨界点近傍（η=0.4）の密スイープ
- **M3（本実験）**: 臨界点近傍（η=0.8）の密スイープ

## 推定時間
~70 runs × 9.5 min ≈ **11 時間**

In [ ]:
# ===== セットアップ =====
from google.colab import drive
drive.mount('/content/drive')

import os
from datetime import datetime

EXP_NAME = 'exp_M3_fine_sweep_eta08'
TIMESTAMP = datetime.now().strftime('%Y%m%d_%H%M%S')
BASE_DIR = '/content/drive/MyDrive/dual-gradient-learning/Paper-A'
SAVE_DIR = f'{BASE_DIR}/{EXP_NAME}_{TIMESTAMP}'
os.makedirs(SAVE_DIR, exist_ok=True)
os.makedirs(f'{SAVE_DIR}/figures', exist_ok=True)

print(f'Experiment: {EXP_NAME}')
print(f'Timestamp: {TIMESTAMP}')
print(f'Save directory: {SAVE_DIR}')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils import parameters_to_vector
import torchvision
import torchvision.transforms as transforms
import numpy as np
import json
import time

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if torch.cuda.is_available():
    torch.backends.cudnn.benchmark = True
    print(f'GPU: {torch.cuda.get_device_name(0)}')
print(f'Device: {device}')

In [ ]:
# ===== モデル定義 =====
class IndexedDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
    def __getitem__(self, idx):
        img, label = self.dataset[idx]
        return img, label, idx
    def __len__(self):
        return len(self.dataset)

class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, in_planes, planes, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, 3, stride, 1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, 3, 1, 1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, 1, stride, bias=False),
                nn.BatchNorm2d(planes))
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        return F.relu(out)

class ResNet18(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.in_planes = 64
        self.conv1 = nn.Conv2d(3, 64, 3, 1, 1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(64, 2, 1)
        self.layer2 = self._make_layer(128, 2, 2)
        self.layer3 = self._make_layer(256, 2, 2)
        self.layer4 = self._make_layer(512, 2, 2)
        self.linear = nn.Linear(512, num_classes)
    def _make_layer(self, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for s in strides:
            layers.append(BasicBlock(self.in_planes, planes, s))
            self.in_planes = planes
        return nn.Sequential(*layers)
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.adaptive_avg_pool2d(out, 1)
        out = out.view(out.size(0), -1)
        return self.linear(out)

In [ ]:
# ===== 実験パラメータ =====
BATCH_SIZE = 256
NUM_WORKERS = 4
EPOCHS = 100
LR = 0.1
K = 16

# M3: 密スイープ（η=0.8）
LAMBDAS = [0.38, 0.40, 0.42, 0.44, 0.46, 0.48, 0.50]
NOISE_RATE = 0.8  # 固定（M2との違い）
SEEDS = list(range(10))  # 0-9

# 実験条件リスト生成
experiments = []
for lam in LAMBDAS:
    for seed in SEEDS:
        experiments.append({
            'lambda': lam,
            'noise_rate': NOISE_RATE,
            'seed': seed
        })

total_runs = len(experiments)
print(f'Lambdas: {LAMBDAS}')
print(f'Noise rate: {NOISE_RATE} (fixed)')
print(f'Seeds: {SEEDS}')
print(f'Total runs: {total_runs}')
print(f'Estimated time: {total_runs * 9.5 / 60:.1f} hours')

# config保存
config = {
    'experiment': EXP_NAME,
    'timestamp': TIMESTAMP,
    'description': 'Fine lambda sweep around critical region (eta=0.8)',
    'parameters': {
        'lambdas': LAMBDAS,
        'noise_rate': NOISE_RATE,
        'seeds': SEEDS,
        'epochs': EPOCHS,
        'batch_size': BATCH_SIZE,
        'lr': LR,
        'K': K
    },
    'total_runs': total_runs,
    'series': 'M (lambda sweep)',
    'part': '3 of 3 (fine, eta=0.8)'
}
with open(f'{SAVE_DIR}/exp_M3_config.json', 'w') as f:
    json.dump(config, f, indent=2)
print(f'Config saved')

In [ ]:
# ===== ユーティリティ関数 =====
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)

def load_cifar10():
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
    testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
    return trainset, testset

def get_data_loaders(trainset, testset):
    indexed_trainset = IndexedDataset(trainset)
    train_loader = DataLoader(indexed_trainset, batch_size=BATCH_SIZE, shuffle=True,
                              num_workers=NUM_WORKERS, pin_memory=True, persistent_workers=True, drop_last=True)
    test_loader = DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False,
                             num_workers=NUM_WORKERS, pin_memory=True, persistent_workers=True)
    return train_loader, test_loader

def inject_noise(labels, noise_rate, seed):
    """ラベルにノイズを注入（seedごとに異なるパターン）"""
    np.random.seed(seed + 1000)
    noisy_labels = labels.copy()
    n_noisy = int(noise_rate * len(labels))
    noisy_indices = np.random.choice(len(labels), n_noisy, replace=False)
    for idx in noisy_indices:
        choices = [i for i in range(10) if i != labels[idx]]
        noisy_labels[idx] = np.random.choice(choices)
    return noisy_labels

def evaluate(model, test_loader):
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device, non_blocking=True), targets.to(device, non_blocking=True)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    return correct / total

In [ ]:
# ===== Dual-Gradient Learning（標準版） =====
def train_dual_gradient(model, train_loader, test_loader, clean_labels, noisy_labels, lam):
    """
    標準的なDual-Gradient Learning。
    
    g_struct = noisy labels の勾配（データに従う力）
    g_value = clean labels の勾配（価値方向）
    g_mix = (1-λ) × ĝ_struct + λ × ĝ_value
    """
    optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4)
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[50, 75], gamma=0.1)
    criterion = nn.CrossEntropyLoss()
    
    clean_labels_tensor = torch.tensor(clean_labels, device=device)
    noisy_labels_tensor = torch.tensor(noisy_labels, device=device)
    
    cached_g_value = None
    global_step = 0
    
    best_acc = 0
    cos_history = []
    history = {'epoch': [], 'test_acc': [], 'test_error': []}
    
    for epoch in range(EPOCHS):
        model.train()
        epoch_cos = []
        
        for inputs, _, indices in train_loader:
            inputs = inputs.to(device, non_blocking=True)
            indices = indices.to(device, non_blocking=True)
            batch_noisy = noisy_labels_tensor[indices]
            batch_clean = clean_labels_tensor[indices]
            
            # Structure gradient (from noisy labels)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss_struct = criterion(outputs, batch_noisy)
            loss_struct.backward(retain_graph=True)
            g_struct = parameters_to_vector([p.grad for p in model.parameters()]).clone()
            
            # Value gradient (from clean labels)
            if global_step % K == 0 or cached_g_value is None:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss_value = criterion(outputs, batch_clean)
                loss_value.backward()
                cached_g_value = parameters_to_vector([p.grad for p in model.parameters()]).clone()
            
            # Normalize and mix
            g_struct_norm = g_struct / (g_struct.norm() + 1e-12)
            g_value_norm = cached_g_value / (cached_g_value.norm() + 1e-12)
            
            cos_sim = (g_struct_norm @ g_value_norm).item()
            epoch_cos.append(cos_sim)
            
            g_mix = (1 - lam) * g_struct_norm + lam * g_value_norm
            
            optimizer.zero_grad()
            idx = 0
            for p in model.parameters():
                numel = p.numel()
                p.grad = g_mix[idx:idx+numel].view(p.shape).clone()
                idx += numel
            optimizer.step()
            global_step += 1
        
        scheduler.step()
        cos_history.append(np.mean(epoch_cos))
        
        if (epoch + 1) % 10 == 0:
            acc = evaluate(model, test_loader)
            best_acc = max(best_acc, acc)
            history['epoch'].append(epoch + 1)
            history['test_acc'].append(acc)
            history['test_error'].append(1 - acc)
    
    final_acc = evaluate(model, test_loader)
    avg_cos = np.mean(cos_history)
    
    return final_acc, max(best_acc, final_acc), avg_cos, history

In [ ]:
# ===== データ準備 =====
trainset, testset = load_cifar10()
clean_labels = np.array(trainset.targets)
train_loader, test_loader = get_data_loaders(trainset, testset)

print('Data prepared')

# GPU warmup
warmup_model = ResNet18().to(device)
for _ in range(20):
    _ = warmup_model(torch.randn(BATCH_SIZE, 3, 32, 32, device=device))
del warmup_model
torch.cuda.empty_cache()
print('Warmup complete.')

In [ ]:
# ===== メイン実験ループ =====
results = []
checkpoint_file = f'{SAVE_DIR}/exp_M3_checkpoint.json'
completed = set()

if os.path.exists(checkpoint_file):
    with open(checkpoint_file, 'r') as f:
        results = json.load(f)
    for r in results:
        completed.add((r['lambda'], r['seed']))
    print(f'Checkpoint loaded: {len(completed)} runs completed')

run_counter = 0
exp_start = time.time()

for exp in experiments:
    run_counter += 1
    lam = exp['lambda']
    noise_rate = exp['noise_rate']
    seed = exp['seed']
    
    key = (lam, seed)
    if key in completed:
        continue
    
    print(f'\n[{run_counter}/{total_runs}] λ={lam:.2f} η={noise_rate} seed={seed}')
    t0 = time.time()
    
    set_seed(seed)
    noisy_labels = inject_noise(clean_labels, noise_rate, seed)
    model = ResNet18().to(device)
    
    final_acc, best_acc, avg_cos, history = train_dual_gradient(
        model, train_loader, test_loader,
        clean_labels, noisy_labels, lam
    )
    elapsed = time.time() - t0
    
    result = {
        'experiment_id': f'M3-{run_counter:03d}',
        'experiment': EXP_NAME,
        'lambda': lam,
        'noise_rate': noise_rate,
        'seed': seed,
        'test_acc': final_acc,
        'test_error': 1 - final_acc,
        'best_acc': best_acc,
        'best_error': 1 - best_acc,
        'avg_cos_struct_value': avg_cos,
        'time_seconds': elapsed,
        'history': history
    }
    results.append(result)
    
    with open(checkpoint_file, 'w') as f:
        json.dump(results, f, indent=2)
    
    status = ''
    if (1 - final_acc) > 0.5:
        status = ' ⚠️ COLLAPSE'
    elif (1 - final_acc) < 0.15:
        status = ' ✅ EXCELLENT'
    
    print(f'  Error: {1-final_acc:.4f} | cos: {avg_cos:.4f} | Time: {elapsed/60:.1f} min{status}')
    
    completed_count = len(results)
    avg_time = sum([r['time_seconds'] for r in results]) / completed_count
    remaining = total_runs - completed_count
    eta_hours = (remaining * avg_time) / 3600
    print(f'  Progress: {completed_count}/{total_runs} | ETA: {eta_hours:.1f} hours')

print(f'\n{"="*60}')
print(f'EXPERIMENT M3 COMPLETE')
print(f'Total time: {(time.time()-exp_start)/3600:.2f} hours')
print(f'{"="*60}')

In [ ]:
# ===== 結果保存 =====
import pandas as pd

with open(f'{SAVE_DIR}/exp_M3_results.json', 'w') as f:
    json.dump(results, f, indent=2)

results_flat = [{k: v for k, v in r.items() if k != 'history'} for r in results]
df = pd.DataFrame(results_flat)
df.to_csv(f'{SAVE_DIR}/exp_M3_results.csv', index=False)

print(f'Results saved to {SAVE_DIR}/')

In [ ]:
# ===== 臨界点分析 =====
import pandas as pd
import numpy as np

df = pd.DataFrame([{k: v for k, v in r.items() if k != 'history'} for r in results])

print('='*70)
print('CRITICAL POINT ANALYSIS (η=0.8)')
print('='*70)

stats = df.groupby('lambda')['test_error'].agg(['mean', 'std', 'count'])
print('\nStatistics by λ:')
print(stats.round(4))

# Variance peak（臨界点候補）
critical_lambda = stats['std'].idxmax()
critical_var = stats['std'].max()
print(f'\n→ Critical λ (max variance): {critical_lambda}')
print(f'   Variance at critical: {critical_var:.4f}')

# 最急勾配点
stats_sorted = stats.sort_index()
gradient = np.gradient(stats_sorted['mean'].values, stats_sorted.index.values)
steepest_idx = np.argmax(np.abs(gradient))
steepest_lambda = stats_sorted.index[steepest_idx]
print(f'\n→ Steepest gradient at: λ = {steepest_lambda}')

In [ ]:
# ===== 可視化 =====
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 左: 相構造
ax = axes[0]
stats = df.groupby('lambda')['test_error'].agg(['mean', 'std'])
ax.errorbar(stats.index, stats['mean'], yerr=stats['std'],
            marker='o', capsize=4, linewidth=2, markersize=8, color='C0')
ax.axvline(x=critical_lambda, color='red', linestyle='--', alpha=0.7,
           label=f'Critical λ = {critical_lambda}')
ax.set_xlabel('λ', fontsize=12)
ax.set_ylabel('Test Error', fontsize=12)
ax.set_title(f'Fine Sweep: η = 80%', fontsize=13, fontweight='bold')
ax.legend()
ax.grid(alpha=0.3)

# 右: Variance（揺らぎ）
ax = axes[1]
ax.plot(stats.index, stats['std'], marker='s', linewidth=2, markersize=8, color='C1')
ax.axvline(x=critical_lambda, color='red', linestyle='--', alpha=0.7,
           label=f'Max variance at λ = {critical_lambda}')
ax.set_xlabel('λ', fontsize=12)
ax.set_ylabel('Variance (std)', fontsize=12)
ax.set_title('Variance Peak (Susceptibility-like)', fontsize=13, fontweight='bold')
ax.legend()
ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig(f'{SAVE_DIR}/figures/exp_M3_critical_analysis.png', dpi=300, bbox_inches='tight')
plt.show()
print(f'Figure saved: {SAVE_DIR}/figures/exp_M3_critical_analysis.png')

In [ ]:
# ===== 分布の可視化 =====
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.flatten()

for i, lam in enumerate(LAMBDAS):
    if i >= len(axes):
        break
    ax = axes[i]
    errors = df[df['lambda'] == lam]['test_error'].values
    
    ax.hist(errors, bins=10, edgecolor='black', alpha=0.7)
    ax.axvline(x=np.mean(errors), color='red', linestyle='--', label=f'mean={np.mean(errors):.3f}')
    ax.set_xlabel('Test Error')
    ax.set_ylabel('Count')
    ax.set_title(f'λ = {lam}', fontweight='bold')
    ax.legend(fontsize=8)

# 余ったaxesを非表示
for j in range(len(LAMBDAS), len(axes)):
    axes[j].set_visible(False)

plt.suptitle('Distribution of Test Errors (η=0.8, 10 seeds each)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(f'{SAVE_DIR}/figures/exp_M3_distributions.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# ===== サマリー =====
print('\n' + '='*70)
print('EXPERIMENT M3: FINE SWEEP (η=0.8) - SUMMARY')
print('='*70)
print(f'\nCritical λ (max variance): {critical_lambda}')
print(f'Steepest gradient at: {steepest_lambda}')
print(f'\nSave directory: {SAVE_DIR}')